In [11]:
using System;
using System.Collections.Concurrent;
using System.Threading;

public interface ICommand
{
    void Execute();
}

public class ServerThread
{
    private BlockingCollection<ICommand> commandQueue = new BlockingCollection<ICommand>();
    private Thread thread;
    private bool isRunning = true;
    private bool isSoftStopping = false;
    private bool isHardStopping = false;
    private ConcurrentQueue<ICommand> schedulerQueue = new ConcurrentQueue<ICommand>();

    public ServerThread()
    {
        thread = new Thread(Run);
        thread.Start();
    }

    public void AddCommand(ICommand command)
    {
        commandQueue.Add(command);
    }

    private void Run()
    {
        while (isRunning)
        {
            ICommand command;
            if (commandQueue.TryTake(out command, Timeout.Infinite))
            {
                if (!isSoftStopping && !isHardStopping)
                {
                    try
                    {
                        command.Execute();
                    }
                    catch (Exception ex)
                    {
                        // Обработка исключения
                        Console.WriteLine($"Исключение в команде: {ex.Message}");
                    }
                }
                else
                {
                    commandQueue.Add(command);
                }
            }
            else if (isSoftStopping)
            {
                isRunning = false;
            }
            else
            {
                Thread.Sleep(1);
            }

            if (schedulerQueue.TryDequeue(out command))
            {
                commandQueue.Add(command);
            }
        }
    }

    public void SoftStop()
    {
        isSoftStopping = true;
    }

    public void HardStop()
    {
        isHardStopping = true;
        thread.Abort();
    }
}

public class LongRunningCommand : ICommand
{
    private readonly int iterations;
    private int currentIteration;
    private readonly ServerThread serverThread;

    public LongRunningCommand(int iterations, ServerThread serverThread)
    {
        this.iterations = iterations;
        this.serverThread = serverThread;
    }

    public void Execute()
    {
        for (int i = 0; i < 10; i++)
        {
            Console.WriteLine($"Команда {GetHashCode()} - итерация {currentIteration + i + 1} из {iterations}");
            Thread.Sleep(100);
        }

        currentIteration += 10;

        if (currentIteration < iterations)
        {
            serverThread.AddCommand(this); // добавление команды в очередь планировщика
        }
    }
}

In [12]:
var serverThread = new ServerThread();

serverThread.AddCommand(new LongRunningCommand(100, serverThread));
serverThread.AddCommand(new LongRunningCommand(50, serverThread));
serverThread.AddCommand(new LongRunningCommand(200, serverThread));

Thread.Sleep(5000);

serverThread.SoftStop();

Console.WriteLine("Основной поток завершается.");


Команда 42331118 - итерация 1 из 100
Команда 42331118 - итерация 2 из 100
Команда 42331118 - итерация 3 из 100
Команда 42331118 - итерация 4 из 100
Команда 42331118 - итерация 5 из 100
Команда 42331118 - итерация 6 из 100
Команда 42331118 - итерация 7 из 100
Команда 42331118 - итерация 8 из 100
Команда 42331118 - итерация 9 из 100
Команда 42331118 - итерация 10 из 100
Команда 11408449 - итерация 1 из 50
Команда 11408449 - итерация 2 из 50
Команда 11408449 - итерация 3 из 50
Команда 11408449 - итерация 4 из 50
Команда 11408449 - итерация 5 из 50
Команда 11408449 - итерация 6 из 50
Команда 11408449 - итерация 7 из 50
Команда 11408449 - итерация 8 из 50
Команда 11408449 - итерация 9 из 50
Команда 11408449 - итерация 10 из 50
Команда 6035355 - итерация 1 из 200
Команда 6035355 - итерация 2 из 200
Команда 6035355 - итерация 3 из 200
Команда 6035355 - итерация 4 из 200
Команда 6035355 - итерация 5 из 200
Команда 6035355 - итерация 6 из 200
Команда 6035355 - итерация 7 из 200
Команда 6035355 